# Important: do Cell > All Outputs > Clear before commiting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from qiskit import BasicAer, QuantumCircuit, ClassicalRegister, QuantumRegister, execute

In [ ]:
NUM_BITS = 3

In [ ]:
x = QuantumRegister(NUM_BITS)
work = QuantumRegister(NUM_BITS-2)
y = QuantumRegister(1)

def u_omega(k):
    qc = QuantumCircuit(x, work, y)
    for i in range(NUM_BITS):
        if k & (1 << i):
            pass
        else:
            qc.x(x[i])
    qc.barrier()
    qc.ccx(x[0], x[1], work[0])
    for i in range(2, NUM_BITS-1):
        qc.ccx(x[i], work[i-2], work[i-1])
    qc.ccx(x[-1], work[-1], y)
    for i in range(NUM_BITS-2, 1, -1):
        qc.ccx(x[i], work[i-2], work[i-1])
    qc.ccx(x[0], x[1], work[0])
    
    qc.barrier()
    for i in range(NUM_BITS):
        if k & (1 << i):
            pass
        else:
            qc.x(x[i])

    return qc

# S = {0,1,4,5}
N = 4
def a():
    qc = QuantumCircuit(x)
    qc.h(x[0])
    qc.h(x[2])
    return qc

def a_inv():
    return a()

def diffusion():
    qc = QuantumCircuit(x)
    qc += a_inv()
    qc += u_omega(0)
    qc += a()
    return qc

def grover_iteration(k):
    qc = QuantumCircuit()
    qc += u_omega(k)
    qc += diffusion()
    return qc

In [ ]:
THETA = 2*np.arcsin(1/np.sqrt(N))
ITERS = (np.pi/2 - THETA/2) / THETA

print(ITERS)
ITERS = int(np.round(ITERS))
print(ITERS)


qc = QuantumCircuit(x,work,y)
qc += a()
qc.x(y)
qc.h(y)
qc.barrier()
for _ in range(1):
    qc += grover_iteration(0)


#qc += u_omega(4)

In [ ]:

# Select the StatevectorSimulator from the Aer provider
simulator = BasicAer.get_backend('statevector_simulator')

# Execute and get counts
result = execute(qc, simulator).result()
statevector = result.get_statevector(qc)
# statevector[bitconcat()]
for i in range(len(statevector)):
    x_val = i & ((1<<NUM_BITS)-1)
    y_val = i >> (NUM_BITS + NUM_BITS-2)
    if statevector[i] != 0.0:
        print("{:.3f} |{}>|{}>".format(statevector[i], x_val, y_val))